# Check GPUs

In [ ]:
! nvidia-smi

# One-time setup

In [ ]:
#!git clone https://github.com/matterport/Mask_RCNN.git

# Definitions

In [ ]:
# Imports
import os
import sys
import itertools
import math
import logging
import json
import re
import random
from collections import OrderedDict
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.lines as lines
from matplotlib.patches import Polygon
import datetime
import skimage.io
from imgaug import augmenters as iaa

# Important directories of the project
ROOT_DIR = os.path.abspath("../../")

DATASET_DIR = os.path.join(ROOT_DIR, "datasets", "satellite")
TRAIN_IMG_DIR = os.path.join(DATASET_DIR, "train_images")
VAL_IMG_DIR = os.path.join(DATASET_DIR, "val_images")
TRAIN_ANNOT_DIR = os.path.join(DATASET_DIR, "mask" , "train_mask_annotations")
VAL_ANNOT_DIR = os.path.join(DATASET_DIR, "mask", "val_mask_annotations")

COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
MODEL_DIR = os.path.join(ROOT_DIR, "logs")


# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")


# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log
from mrcnn.config import Config
from mrcnn import model


#import satellite

%matplotlib inline 

In [ ]:
import keras
print(keras.__version__)
import tensorflow
print(tensorflow.__version__)
import skimage
print(skimage.__version__)

In [ ]:
!nvidia-smi

In [ ]:
# print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))

In [ ]:
############################################################
#  Configurations
############################################################

class SatelliteConfig(Config):
    """Configuration for training on the satellite segmentation dataset."""
    # Give the configuration a recognizable name
    NAME = "satellite"

    # Adjust depending on your GPU memory
    IMAGES_PER_GPU = 2

    GPU_COUNT = 2
    
    # Number of classes (including background)
    NUM_CLASSES = 1 + 3  # Background + body + solar_panel + antenna

    # Number of training and validation steps per epoch
    STEPS_PER_EPOCH = 1000

    #DETECTION_MIN_CONFIDENCE = 0.5


class SatelliteInferenceConfig(SatelliteConfig):
    # Set batch size to 1 to run one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    # Don't resize imager for inferencing
    IMAGE_RESIZE_MODE = "none"
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    RPN_NMS_THRESHOLD = 0.7


In [ ]:
############################################################
#  Dataset
############################################################

class SatelliteDataset(utils.Dataset):

    def load_satellite(self, dataset_type_dir, mask_dir):

        self.add_class("satellite", 1, "body")
        self.add_class("satellite", 2, "solar_panel")
        self.add_class("satellite", 3, "antenna")

        file_names = os.listdir(dataset_type_dir)
        image_ids = []
        for file_name in file_names:
            if file_name.endswith(".png"):
                image_ids.append(file_name)
        #print(image_ids)
        
        for image_id in image_ids:
            temp_str = image_id.replace('.png','')
            temp_str = temp_str + '_mask'
            mask_dir_img_id = os.path.join(mask_dir, temp_str)
            #print(mask_dir_img_id)
            self.add_image(
                "satellite",
                image_id=image_id,
                path=os.path.join(dataset_type_dir, image_id),
                mask_dir_img_id = mask_dir_img_id
                )

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        info = self.image_info[image_id]
        # Get mask directory from image path
        mask_dir_img_id = info['mask_dir_img_id']
        #mask_dir = os.path.join(os.path.dirname(os.path.dirname(info['path'])), "masks")
        # Read mask files from .png image
        #print(image_id, mask_dir_img_id)
        mask = []
        class_ids = []

        # mask_dir_img_id_list = os.listdir(mask_dir_img_id)
        for f in next(os.walk(mask_dir_img_id))[2]:
        # for f in mask_dir_img_id_list:
            if f.endswith(".png"):
                if 'body' in f:
                    class_ids.append(1)
                if 'solar_panel' in f:
                    class_ids.append(2)
                if 'antenna' in f:
                    class_ids.append(3)
                m = skimage.io.imread(os.path.join(mask_dir_img_id, f)).astype(np.bool)
                mask.append(m)
        mask = np.stack(mask, axis=-1)
        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID, we return an array of ones
        #------------------------------------------------------------

        return mask, np.array(class_ids)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "satellite":
            return info["id"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [ ]:
config = SatelliteConfig()
config.display()

In [ ]:
dataset_train = SatelliteDataset()
dataset_train.load_satellite(TRAIN_IMG_DIR, TRAIN_ANNOT_DIR)

# Must call before using the dataset
dataset_train.prepare()

print("Image Count: {}".format(len(dataset_train.image_ids)))
print("Class Count: {}".format(dataset_train.num_classes))
for i, info in enumerate(dataset_train.class_info):
    print("{:3}. {:50}".format(i, info['name']))

In [ ]:
dataset_val = SatelliteDataset()
dataset_val.load_satellite(VAL_IMG_DIR, VAL_ANNOT_DIR)
# Must call before using the dataset
dataset_val.prepare()

print("Image Count: {}".format(len(dataset_val.image_ids)))
print("Class Count: {}".format(dataset_val.num_classes))
for i, info in enumerate(dataset_val.class_info):
    print("{:3}. {:50}".format(i, info['name']))

# Inspect Data

In [ ]:
# Load and display random samples
#image_ids = dataset.image_ids
image_ids = np.random.choice(dataset_train.image_ids, 7)
#print(image_ids)
for image_id in image_ids:
    image = dataset_train.load_image(image_id)
    print(image_id)
    #plt.imshow(image)
    mask, class_ids = dataset_train.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

In [ ]:
# Load and display random samples
#image_ids = dataset.image_ids
image_ids = np.random.choice(dataset_val.image_ids, 7)
#print(image_ids)
for image_id in image_ids:
    image = dataset_val.load_image(image_id)
    print(image_id)
    #plt.imshow(image)
    mask, class_ids = dataset_val.load_mask(image_id)
    visualize.display_top_masks(image, mask, class_ids, dataset_val.class_names)

# Create Model & Train

In [ ]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

In [ ]:
init_with = "last"  

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
  
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    last_saved_model = model.find_last()
    print(last_saved_model)
    model.load_weights(last_saved_model, by_name=True)

In [ ]:
model.train(dataset_train, dataset_val,
            learning_rate=config.LEARNING_RATE,
            epochs=40,
            layers='all')

In [ ]:
cur_dir = os.getcwd()

In [ ]:
# Inference Configuration
config_infer = SatelliteInferenceConfig()
config_infer.display()

In [ ]:
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# Only inference mode is supported right now
TEST_MODE = "inference"

In [ ]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    fig, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    fig.tight_layout()
    return ax

In [ ]:
# Create model in inference mode
with tensorflow.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference",
                              model_dir=MODEL_DIR,
                              config=config_infer)

In [ ]:
os.getcwd()

In [ ]:
# Path to a specific weights file
# weights_path = os.path.join(ROOT_DIR, "mask_rcnn_satellite_0025.h5")

# Or, load the last model you trained
weights_path = model.find_last()

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

In [ ]:
image_id = random.choice(dataset_val.image_ids)
image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_val, config_infer, image_id, use_mini_mask=False)
info = dataset_val.image_info[image_id]
print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, 
                                       dataset_val.image_reference(image_id)))
print("Original image shape: ", modellib.parse_image_meta(image_meta[np.newaxis,...])["original_image_shape"][0])

# Run object detection
results = model.detect_molded(np.expand_dims(image, 0), np.expand_dims(image_meta, 0), verbose=1)

# Display results
r = results[0]
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

# Compute AP over range 0.5 to 0.95 and print it
utils.compute_ap_range(gt_bbox, gt_class_id, gt_mask,
                       r['rois'], r['class_ids'], r['scores'], r['masks'],
                       verbose=1)

visualize.display_differences(
    image,
    gt_bbox, gt_class_id, gt_mask,
    r['rois'], r['class_ids'], r['scores'], r['masks'],
    dataset.class_names, ax=get_ax(),
    show_box=False, show_mask=False,
    iou_threshold=0.5, score_threshold=0.5)

# Plot Training Graph

In [ ]:
import datetime
%load_ext tensorboard

In [ ]:
%pwd

In [ ]:
%tensorboard --logdir path/to/log --port=6006

# Inferencing

In [ ]:
class SatelliteInferenceConfig(SatelliteConfig):
    # Set batch size to 1 to run one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

In [ ]:
# Inference Configuration
config_infer = SatelliteInferenceConfig()
# config_infer.display()

In [ ]:
DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0

TEST_MODE = "inference"

In [ ]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
# Create model in inference mode
with tensorflow.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config_infer)

In [ ]:
weights_path = "path/to/trained_model"
# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

## Inference on validation dataset

In [ ]:
dataset_val = SatelliteDataset()
dataset_val.load_satellite(VAL_IMG_DIR, VAL_ANNOT_DIR)
# Must call before using the dataset
dataset_val.prepare()

print("Image Count: {}".format(len(dataset_val.image_ids)))
print("Class Count: {}".format(dataset_val.num_classes))
for i, info in enumerate(dataset_val.class_info):
    print("{:3}. {:50}".format(i, info['name']))

In [ ]:
image_id = random.choice(dataset_val.image_ids)
print(image_id)
image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_val, config_infer, image_id, use_mini_mask=False)
info = dataset_val.image_info[image_id]
print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, 
                                       dataset_val.image_reference(image_id)))


# Run object detection
results = model.detect([image], verbose=1)

# Display results
ax = get_ax(1)
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=ax,
                            title="Predictions")
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

In [ ]:
mask_tmp0 = r['masks']
# print(mask_tmp0, type(mask_tmp0))
print(mask_tmp0.shape, mask_tmp0.size)
print(image.shape, image.size)


In [ ]:
fig, axis = plt.subplots(2,2, figsize = (16,6))

axis[0, 0].imshow(mask_tmp0[:,:,0])
axis[0, 1].imshow(mask_tmp0[:,:,1])
axis[1, 0].imshow(mask_tmp0[:,:,2])
axis[1, 1].imshow(mask_tmp0[:,:,3])

plt.tight_layout()


In [ ]:
import cv2
mask_predi = mask_tmp0[:,:,0] + mask_tmp0[:,:,1] + mask_tmp0[:,:,2] +  mask_tmp0[:,:,3]
# mask_predi = cv2.cvtColor(mask_predi, cv2.COLOR_BGR2RGB)
plt.imshow(mask_predi, cmap = 'gray')

## Inference on training dataset

In [ ]:
dataset_train = SatelliteDataset()
dataset_train.load_satellite(TRAIN_IMG_DIR, TRAIN_ANNOT_DIR)

# Must call before using the dataset
dataset_train.prepare()

print("Image Count: {}".format(len(dataset_train.image_ids)))
print("Class Count: {}".format(dataset_train.num_classes))
for i, info in enumerate(dataset_train.class_info):
    print("{:3}. {:50}".format(i, info['name']))

In [ ]:
image_id = random.choice(dataset_train.image_ids)
print(image_id)
image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_train, config_infer, image_id, use_mini_mask=False)
info = dataset_train.image_info[image_id]
print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, 
                                       dataset_train.image_reference(image_id)))

# Run object detection
results = model.detect([image], verbose=1)

# Display results
ax = get_ax(1)
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=ax,
                            title="Predictions")
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

## Custom inference

In [ ]:
weights_path = "/path/to/trained_model"
# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

In [ ]:
# Run object detection on own images
import cv2
sat_test_img = os.path.join(ROOT_DIR, "sat_test_img", "0_26_hr.png")
# sat_test_img = cv2.resize(sat_test_img, (1024, 1024))
# image = plt.imread(sat_test_img)
image = cv2.imread(sat_test_img)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

results = model.detect([image], verbose=1)

# Display results
ax = get_ax(1)
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=ax,
                            title="Predictions")

# log("gt_class_id", gt_class_id)
# log("gt_bbox", gt_bbox)
# log("gt_mask", gt_mask)

###### Single run

In [1]:
!pwd
import cv2

def Comb_Masks(masks_all):
    
    # print(masks_all, type(masks_all))
    print(masks_all.shape, masks_all.size, masks_all.dtype)
    print(image.shape, image.size, image.dtype)

    W, H, N_m = masks_all.shape
    # print(W, H, N_m)

    masks_comb = np.zeros((W,H), dtype = bool)
    # print(mask_comb)
    for i in range(N_m):
        masks_comb = masks_comb + masks_all[:,:,i]

    return masks_comb

masks_all = r['masks']
masks_comb = Comb_Masks(masks_all)
plt.imsave('mask_pred_1028_teeeest.png', masks_comb, cmap = 'gray')
# cv2.imwrite('mask_pred_1028_teeeest.png', masks_comb)


/media/inx/data_partition/Projects/ODiS/Space-Intelligence-Vision-and-Analytics/segmentation/Mask_RCNN


NameError: name 'r' is not defined

###### Dataset run

In [ ]:
dataset_val = SatelliteDataset()
dataset_val.load_satellite(VAL_IMG_DIR, VAL_ANNOT_DIR)
# Must call before using the dataset
dataset_val.prepare()

print("Image Count: {}".format(len(dataset_val.image_ids)))
print("Class Count: {}".format(dataset_val.num_classes))
for i, info in enumerate(dataset_val.class_info):
    print("{:3}. {:50}".format(i, info['name']))

In [ ]:
os.getcwd()

In [ ]:
os.chdir('val_masks_pred_BnW')

In [ ]:
# cwd = os.getcwd()
# cwd
%cd Val_masks_pred_BnW

In [ ]:
os.getcwd()

In [ ]:
!pwd
import cv2
import os

cwd = os.getcwd()

def Comb_Masks(masks_all, image_name):
    
    # print(masks_all, type(masks_all))
    print(masks_all.shape, masks_all.size, masks_all.dtype)
    print(image.shape, image.size, image.dtype)

    W, H, N_m = masks_all.shape
    # print(W, H, N_m)

    masks_comb = np.zeros((W,H), dtype = bool)
    # print(mask_comb)
    for i in range(N_m):
        masks_comb = masks_comb + masks_all[:,:,i]
        
    image_name = image_name.replace('.png','')
    plt.imsave(f'{image_name}_mask.png', masks_comb, cmap = 'gray')
    
    return None



In [ ]:
str1 = "something.png"
print(str1)
# print(str1 - '.png')
str1 = str1.replace('.png','')
print(str1)
str1_1 = str1 + '_mask.png'
print(str1_1)
#or
str1_2 = f"{str1}_mask.png"
print(str1_2)


In [ ]:
# info = dataset_val.image_info[image_id]
# print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, 
#                                        dataset_val.image_reference(image_id)))
for image_id in dataset_val.image_ids:

    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, config_infer, image_id, use_mini_mask=False)
    info = dataset_val.image_info[image_id]
    
    results = model.detect([image], verbose=1)

    ax = get_ax(1)
    r = results[0]
    masks_all = r['masks']
    
    image_name = info["id"]
    Comb_Masks(masks_all, image_name)


In [ ]:
# Whole folder
VAL_IMG_DIR_list = os.listdir(VAL_IMG_DIR)
len(VAL_IMG_DIR_list)

In [ ]:
import cv2
for image_name in VAL_IMG_DIR_list:

    image_path = os.path.join(VAL_IMG_DIR, image_name)
    image = cv2.imread(image_path)

    results = model.detect([image], verbose=1)

    # Display results
    ax = get_ax(1)
    r = results[0]
    masks_all = r['masks']
    Comb_Masks(masks_all, image_name)

In [ ]:
!pwd

import cv2

mask_pred = mask_tmp0[:,:,0] + mask_tmp0[:,:,1] + mask_tmp0[:,:,2]   # True/False addition
print(mask_pred.shape)
print(mask_pred)
# mask_predi = cv2.cvtColor(mask_predi, cv2.COLOR_BGR2RGB)
plt.imsave('mask_pred_1028.png', mask_predi, cmap = 'gray')
plt.imshow(mask_predi, cmap = 'gray')

In [ ]:
def display_instances(image, boxes, masks, class_ids, class_names,
                      scores=None, title="",
                      figsize=(16, 16), ax=None,
                      show_mask=True, show_bbox=True,
                      colors=None, captions=None):
    """
    boxes: [num_instance, (y1, x1, y2, x2, class_id)] in image coordinates.
    masks: [height, width, num_instances]
    class_ids: [num_instances]
    class_names: list of class names of the dataset
    scores: (optional) confidence scores for each box
    title: (optional) Figure title
    show_mask, show_bbox: To show masks and bounding boxes or not
    figsize: (optional) the size of the image
    colors: (optional) An array or colors to use with each object
    captions: (optional) A list of strings to use as captions for each object
    """
    # Number of instances
    N = boxes.shape[0]
    if not N:
        print("\n*** No instances to display *** \n")
    else:
        assert boxes.shape[0] == masks.shape[-1] == class_ids.shape[0]

    # If no axis is passed, create one and automatically call show()
    auto_show = False
    if not ax:
        _, ax = plt.subplots(1, figsize=figsize)
        auto_show = True

    # Generate random colors
    colors = colors or visualize.random_colors(N)

    # Show area outside image boundaries.
    height, width = image.shape[:2]
    ax.set_ylim(height + 10, -10)
    ax.set_xlim(-10, width + 10)
    ax.axis('off')
    ax.set_title(title)

    masked_image = image.astype(np.uint32).copy()
    for i in range(N):
        color = colors[i]

        # Bounding box
        if not np.any(boxes[i]):
            # Skip this instance. Has no bbox. Likely lost in image cropping.
            continue
        y1, x1, y2, x2 = boxes[i]
        if show_bbox:
            p = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2,
                                alpha=0.7, linestyle="dashed",
                                edgecolor=color, facecolor='none')
            ax.add_patch(p)

        # Label
        if not captions:
            class_id = class_ids[i]
            score = scores[i] if scores is not None else None
            label = class_names[class_id]
            caption = "{} {:.3f}".format(label, score) if score else label
        else:
            caption = captions[i]
        ax.text(x1, y1 + 8, caption,
                color='w', size=11, backgroundcolor="none")

        # Mask
        mask = masks[:, :, i]
        if show_mask:
            masked_image = visualize.apply_mask(masked_image, mask, color)

        # Mask Polygon
        # Pad to ensure proper polygons for masks that touch image edges.
        padded_mask = np.zeros(
            (mask.shape[0] + 2, mask.shape[1] + 2), dtype=np.uint8)
        padded_mask[1:-1, 1:-1] = mask
        contours = visualize.find_contours(padded_mask, 0.5)
        for verts in contours:
            # Subtract the padding and flip (y, x) to (x, y)
            verts = np.fliplr(verts) - 1
            p = Polygon(verts, facecolor="none", edgecolor=color)
            ax.add_patch(p)
        return masked_image.astype(np.uint8)

In [ ]:
# Run object detection on own images

sat_test_img = os.path.join(ROOT_DIR, "sat_test_img", "sat_img_12.jpg")
image = plt.imread(sat_test_img)

results = model.detect([image], verbose=1)

# Display results
ax = get_ax(1)
r = results[0]
# masked_image = 
display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=ax,
                            title="Predictions")

# plt.imshow(masked_image)
# plt.imsave("infer0.png", masked_image)
# log("gt_class_id", gt_class_id)
# log("gt_bbox", gt_bbox)
# log("gt_mask", gt_mask)

In [ ]:
class SatelliteInferenceConfig(SatelliteConfig):
    # Set batch size to 1 to run one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    # Don't resize imager for inferencing
    #IMAGE_RESIZE_MODE = "none"
    # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
    #RPN_NMS_THRESHOLD = 0.7


In [ ]:
# Inference Configuration
config_infer = SatelliteInferenceConfig()
config_infer.display()

In [ ]:
# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

In [ ]:
dataset_val = SatelliteDataset()
dataset_val.load_satellite(VAL_IMG_DIR, VAL_ANNOT_DIR)
# Must call before using the dataset
dataset_val.prepare()

print("Image Count: {}".format(len(dataset_val.image_ids)))
print("Class Count: {}".format(dataset_val.num_classes))
for i, info in enumerate(dataset_val.class_info):
    print("{:3}. {:50}".format(i, info['name']))
    

In [ ]:
import cv2

batch_size = 1
# Create model in inference mode
with tensorflow.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config_infer)
    
weights_path = "/home/deepesh/ODiS_Mask_RCNN/Mask_RCNN/logs/satellite20220310T0032/mask_rcnn_satellite_0024.h5"
# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)
CLASS_NAMES_MASKRCNN = ['body','solar_panel','antenna']

In [ ]:
def get_color_dict():
    all_colors = visualize.random_colors(3)
    color_dict = {}
    i = 0
    for c in CLASS_NAMES_MASKRCNN:
        if not c in color_dict:
            color_dict[c] = all_colors[i]
            i = i+1
    color_dict["background"] = (0,0,0)
    return color_dict

COLOR_MAP = get_color_dict()

In [ ]:
def label_to_color_image(labels):
    # Adds color defined by the dataset colormap to the label.
    h,w = labels.shape
    img = np.zeros([h,w,3])
    img = np.zeros((h,w),dtype=(float,3))
    for i in range(h):
        for j in range(w):
            img[i][j] = np.array(COLOR_MAP[CLASS_NAMES_MASKRCNN[labels[i][j]]])        
    img = img*255
    return img.astype(np.uint8)

def combine_masks(img, result):
    boxes = result['rois']
    masks = result['masks']
    class_ids = result['class_ids']
  
    N = boxes.shape[0]
    h,w,c = img.shape
    seg_map = np.zeros((h,w))
    for i in range(N):
        mask = masks[:, :, i]
        mask = mask.astype(np.uint8)
        seg_map = seg_map + mask*class_ids[i]
  
    return seg_map.astype(np.uint8)

def merge_images(foreground, background, alpha=0.3):
    out_img = np.zeros(background.shape,dtype=background.dtype)
    out_img[:,:,:] = (alpha * background[:,:,:]) + ((1-alpha) * foreground[:,:,:])
    return out_img

############################################################################################################################

def display_instances(image, boxes, masks, class_ids, class_names,
                      scores=None, title="",
                      figsize=(16, 16), ax=None,
                      show_mask=True, show_bbox=True,
                      colors=None, captions=None):
    """
    boxes: [num_instance, (y1, x1, y2, x2, class_id)] in image coordinates.
    masks: [height, width, num_instances]
    class_ids: [num_instances]
    class_names: list of class names of the dataset
    scores: (optional) confidence scores for each box
    title: (optional) Figure title
    show_mask, show_bbox: To show masks and bounding boxes or not
    figsize: (optional) the size of the image
    colors: (optional) An array or colors to use with each object
    captions: (optional) A list of strings to use as captions for each object
    """
    # Number of instances
    N = boxes.shape[0]
    if not N:
        print("\n*** No instances to display *** \n")
    else:
        assert boxes.shape[0] == masks.shape[-1] == class_ids.shape[0]

    # If no axis is passed, create one and automatically call show()
    auto_show = False
    if not ax:
        _, ax = plt.subplots(1, figsize=figsize)
        auto_show = True

    # Generate random colors
    colors = colors or visualize.random_colors(N)

    # Show area outside image boundaries.
    height, width = image.shape[:2]
    ax.set_ylim(height + 10, -10)
    ax.set_xlim(-10, width + 10)
    ax.axis('off')
    ax.set_title(title)

    masked_image = image.astype(np.uint32).copy()
    for i in range(N):
        color = colors[i]

        # Bounding box
        if not np.any(boxes[i]):
            # Skip this instance. Has no bbox. Likely lost in image cropping.
            continue
        y1, x1, y2, x2 = boxes[i]
        if show_bbox:
            p = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=2,
                                alpha=0.7, linestyle="dashed",
                                edgecolor=color, facecolor='none')
            ax.add_patch(p)

        # Label
        if not captions:
            class_id = class_ids[i]
            score = scores[i] if scores is not None else None
            label = class_names[class_id]
            caption = "{} {:.3f}".format(label, score) if score else label
        else:
            caption = captions[i]
        ax.text(x1, y1 + 8, caption,
                color='w', size=11, backgroundcolor="none")

        # Mask
        mask = masks[:, :, i]
        if show_mask:
            masked_image = visualize.apply_mask(masked_image, mask, color)

        # Mask Polygon
        # Pad to ensure proper polygons for masks that touch image edges.
        padded_mask = np.zeros(
            (mask.shape[0] + 2, mask.shape[1] + 2), dtype=np.uint8)
        padded_mask[1:-1, 1:-1] = mask
        contours = visualize.find_contours(padded_mask, 0.5)
        for verts in contours:
            # Subtract the padding and flip (y, x) to (x, y)
            verts = np.fliplr(verts) - 1
            p = Polygon(verts, facecolor="none", edgecolor=color)
            ax.add_patch(p)
        return masked_image

def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

def get_masked_image(image, result):
    """
    Applies masks from the results to the given image
  
    """
#     boxes = result['rois']
#     masks = result['masks']
  
#     N = boxes.shape[0]
#     if not N:
#         print("\n*** No instances to display *** \n")

    # colors = visualize.random_colors(N)
    # masked_image = image.astype(np.uint32).copy()

#     for i in range(N):
#         color = colors[i]

#         # Mask
#         mask = masks[:, :, i]
#         # masked_image = visualize.apply_mask(masked_image, mask, color)
    ax = get_ax(1)
    r = result
    masked_image = display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                        dataset_val.class_names, r['scores'], ax=ax,)
    
    return masked_image.astype(np.uint8)

def print_fps(video):
    # Find OpenCV version
    (major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')

    if int(major_ver)  < 3 :
        fps = video.get(cv2.cv.CV_CAP_PROP_FPS)
        print("Frames per second using video.get(cv2.cv.CV_CAP_PROP_FPS): {0}".format(fps))
    else :
        fps = video.get(cv2.CAP_PROP_FPS)
        print("Frames per second using video.get(cv2.CAP_PROP_FPS) : {0}".format(fps))


def make_video(outvid, images=None, fps=30, size=None,
               is_color=True, format="FMP4"):
    """
    Create a video from a list of images.
 
    @param      outvid      output video
    @param      images      list of images to use in the video
    @param      fps         frame per second
    @param      size        size of each frame
    @param      is_color    color
    @param      format      see http://www.fourcc.org/codecs.php
    @return                 see http://opencv-python-tutroals.readthedocs.org/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html
 
    The function relies on http://opencv-python-tutroals.readthedocs.org/en/latest/.
    By default, the video will have the size of the first image.
    It will resize every image to this size before adding them to the video.
    """
    from cv2 import VideoWriter, VideoWriter_fourcc, imread, resize
    fourcc = VideoWriter_fourcc(*format)
    vid = None
    for image in images:
        if not os.path.exists(image):
            raise FileNotFoundError(image)
        img = imread(image)
        if vid is None:
            if size is None:
                size = img.shape[1], img.shape[0]
            vid = VideoWriter(outvid, fourcc, float(fps), size, is_color)
        if size[0] != img.shape[1] and size[1] != img.shape[0]:
            img = resize(img, size)
        vid.write(img)
    vid.release()
    return vid

In [ ]:
VIDEO_DIR = os.path.join(ROOT_DIR, "sat_test_vid")
VIDEO_SAVE_DIR = os.path.join(VIDEO_DIR, "sat_infer_vid")

try:
    if not os.path.exists(VIDEO_SAVE_DIR):
        os.makedirs(VIDEO_SAVE_DIR)
except OSError:
    print ('Error: Creating directory of data')

# os.chdir('./sat_test_img')

# from google.colab import files
# uploads = files.upload()

# os.chdir('../')

In [ ]:
capture = cv2.VideoCapture(os.path.join(VIDEO_DIR, "sat_vid_1.mp4"))

print_fps(capture)

try:
    if not os.path.exists(VIDEO_SAVE_DIR):
        os.makedirs(VIDEO_SAVE_DIR)
except OSError:
    print ('Error: Creating directory of data')
frames = []
frame_count = 0

while True:
    ret, frame = capture.read()
    # Bail out when the video file ends
    if not ret:
        break

    # Save each frame of the video to a list
    frame_count += 1
    frames.append(frame)
    print('frame_count :{0}'.format(frame_count))
    if len(frames) == batch_size:
        results = model.detect(frames, verbose=0)
        # print(results)
        # print(results[0])
        # print(len(results))
        print('Predicted')
        for i, item in enumerate(zip(frames, results)):
            frame = item[0]
            r = item[1]
            # print(r)
            #seg_map = combine_masks(frame, r)
            #seg_image = label_to_color_image(seg_map)
            #frame = merge_images(seg_image, frame)
            frame = get_masked_image(frame, r)
            name = '{0}.jpg'.format(frame_count + i - batch_size)
            name = os.path.join(VIDEO_SAVE_DIR, name)
            print(type(frame), frame.size, frame.shape)
            plt.imshow(frame)
            plt.imsave(name, frame)
            # cv2.imwrite(name, frame)
        # Clear the frames array to start the next batch
        frames = []

capture.release()

In [ ]:
import glob
import os

images = list(glob.iglob(os.path.join(VIDEO_SAVE_DIR, '*.*')))
# Sort the images by integer index
images = sorted(images, key=lambda x: float(os.path.split(x)[1][:-3]))

outvid = os.path.join(VIDEO_DIR, "final.mp4")
make_video(outvid, images, fps=20)